# Packages

In [1]:
import networkx as nx
import random
from operator import itemgetter
from collections import Counter
import inspect
import matplotlib.pyplot as plt
import numpy.linalg as ln
import math
import argparse
from scipy.sparse import diags
import os
from datetime import datetime
from tqdm import tqdm
import statistics
from collections import defaultdict
import time 

# torch packages 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

# Graph Reader

In [56]:
def graph_read(graph_name):
    print("\nGraph {} reading ... ".format(graph_name))
    if graph_name == 'acm':
        adj_mat = np.load(r'./dataset/acm/acm_adj.npy')
        labels  = np.load(r'./dataset/acm/acm_label.npy')
    if graph_name == 'cora':
        adj_mat = np.load(r'./dataset/cora/cora_adj.npy')
        labels  = np.load(r'./dataset/cora/cora_label.npy')
    if graph_name == 'citeseer':
        adj_mat = np.load(r'./dataset/citeseer/citeseer_adj.npy')
        labels  = np.load(r'./dataset/citeseer/citeseer_label.npy')
    if graph_name == 'wiki':
        adj_mat = np.load(r'./dataset/wiki/wiki_adj.npy')
        labels  = np.load(r'./dataset/wiki/wiki_label.npy')
    if graph_name == 'amap':
        adj_mat = np.load(r'./dataset/amap/amap_adj.npy')
        labels  = np.load(r'./dataset/amap/amap_label.npy')
        
    print("Get information of graph {}.\n".format(graph_name))
    
    return nx.from_numpy_array(adj_mat), labels


# Model Construction

In [3]:
class MultiLayerNet(nn.Module):
    def __init__(self, input_size, num_classes, dropout_rate=0.1):
        super(MultiLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):

        out = self.relu(self.bn1(self.fc1(x)))
        out = self.dropout(out)
        out = self.relu(self.bn2(self.fc2(out)))
        out = self.dropout(out)

        out = self.fc3(out)
        F.log_softmax(out, dim=1)

        return out

    
def model_train(model, inputs, labels):
        
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.05)
    
    num_epochs = 200  
    for epoch in range(num_epochs):
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
#         if (epoch+1) % 50 == 0:
#             print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}，Accuracy: {train_acc(outputs, labels):.2f}')
            
    return    


def train_acc(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    
    # Compare the predicted labels with the ground truth (true labels)
    correct = (predicted == labels).sum().item()
    
    # Calculate the accuracy by dividing the number of correct predictions by the total number of samples
    total = labels.size(0)
    accuracy = correct / total
    
    return accuracy


def accuracy(model, inputs, labels):
    outputs = model(inputs)
    # Get the predicted labels (class indices) by taking the argmax along the output dimension
    _, predicted = torch.max(outputs, 1)
    
    # Compare the predicted labels with the ground truth (true labels)
    correct = (predicted == labels).sum().item()
    
    # Calculate the accuracy by dividing the number of correct predictions by the total number of samples
    total = labels.size(0)
    accuracy = correct / total
#     print('Test accuracy:{:.2f}'.format(accuracy))
    
    return accuracy
    
    

# Node Order

In [4]:
def ordered_list_centrality(graph, centrality_type):
    
    nodes = list(graph.nodes())
    
    if centrality_type == 'random':
        print('\nNodes sorted by Random:')
        random.shuffle(nodes)
        return nodes

    if centrality_type == 'lc':
        print('\nNodes sorted by Local Neighbors:')
        return  ordered_by_lc(graph, centrality_type = 'lc')
    
    # types of centrality measurement 
    centrality_measures = {
        'pr': [nx.pagerank               , 'PageRank'],
        'dg': [nx.degree_centrality      , 'Degree Centrality'],
        'bt': [nx.betweenness_centrality , 'Betweenness Centrality'],
        'cl': [nx.closeness_centrality   , 'Closeness Centrality'],
        'eg': [nx.eigenvector_centrality , 'Eigenvector Centrality'] 
    }
    # 字典 key-value : 节点-中心性
    node_centrality = centrality_measures[centrality_type][0](graph)
    
    # 对 centrality 值进行排序
    sorted_centrality = sorted(node_centrality.items(), key=lambda x: x[1], reverse=True)
     
    # 获取排序后的节点列表
    sorted_nodes = [node for node, rank in sorted_centrality]

    print('\nNodes sorted by {}:'.format(centrality_measures[centrality_type][1]))

    return sorted_nodes


def ordered_list_centrality_probability(graph, centrality_type):
    
    nodes = list(graph.nodes())
    
    if centrality_type == 'random':
        print('Nodes sorted by Random\n')
        random.shuffle(nodes)
        return nodes

    # types of centrality measurement 
    centrality_measures = {
        'pr': [nx.pagerank               , 'PageRank'],
        'dg': [nx.degree_centrality      , 'Degree Centrality'],
        'bt': [nx.betweenness_centrality , 'Betweenness Centrality'],
        'cl': [nx.closeness_centrality   , 'Closeness Centrality'],
        'eg': [nx.eigenvector_centrality , 'Eigenvector Centrality'] 
    }
    # 字典 key-value : 节点-中心性
    node_centrality = centrality_measures[centrality_type][0](graph)
    
    ## 依概率排序
    # 获取点的数量
    num_points = len(nodes)
    # 获取rank值总和
    total_rank = sum(node_centrality.values())
    # 计算每个点作为首位的概率
    probabilities = {point: rank / total_rank for point, rank in node_centrality.items()}
    # 生成排序后的点列表
    sorted_nodes = sorted(probabilities, key=lambda x: random.random() ** (1/(probabilities[x] if probabilities[x] != 0 else 1)), reverse=True)

    print('\nNodes sorted by {}:'.format(centrality_measures[centrality_type][1]))

    return sorted_nodes


def ordered_by_lc(graph, centrality_type = 'lc'):
    
#     print('Select nodes by set cover on local clusters.')
    nodes = list(graph.nodes())
    epsilon = 10 ** - 2.5
    node_local_cluster = defaultdict(list)
    for node in nodes:
        vec = forward_search(graph, node, epsilon)
#         vec = backward_search(graph, node, epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0]
        node_local_cluster[node] = non_zero_indices
        
    selected_nodes = set_cover(node_local_cluster, 250)
    
    return list(selected_nodes)


def set_cover(graph, K):
    
    universe = set.union(*map(set, graph.values()))

    
    selected_nodes = set()
    iterations = 0

    while universe and iterations < K:
        
        node_coverages = {node: len(set(universe).intersection(nodes)) for node, nodes in graph.items()}
        selected_node = max(node_coverages, key=node_coverages.get)       
        selected_nodes.add(selected_node)
        universe -= set(graph[selected_node])
      
        del graph[selected_node]

        iterations += 1
        
    print(len(universe))
        
    return selected_nodes


def order_test():
    
    graph, _ = graph_read('cora')
#     ordered_list = ordered_list_centrality(graph, 'eg')
    
    ordered_list = ordered_by_lc(graph, centrality_type = 'lc')
    
    return


# order_test()

## Anchor Nodes Selection

In [44]:
def marginal_score(k, anchors: [int]) -> int:

    print(k, anchors, sep = ': ')
    
    return 0


marginal_score(3, [1, 2])


def anchor_nodes_v1(graph, topk, k) -> []:
   
    nodes = list(graph.nodes())
    epsilon = 10 ** -2.5
    node_local_cluster = defaultdict(list)
    for node in nodes:
        vec = forward_search(graph, node, epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0]
        node_local_cluster[node] = non_zero_indices            # indices of radiated neighbors
    
    start_time = time.time()                     
    node_actnum = {}
    node_mscore:[(int, int)] =  []               # [(node, mscore)]
    for node in nodes: 
#         node_actnum[node] = 0
        node_mscore.append((node, k*len(node_local_cluster[node]))) # initial marginal score for each node
    
    anchors = []
    node_mscore.sort(key = lambda x: x[1], reverse = True)
    anchors.append(node_mscore[0][0])

#     radiated_nodes = []   
    
    for i in range(1, topk):
        
        for rad_node in node_local_cluster[anchors[-1]]:              
            actnum = node_actnum.get(rad_node, 0)+1          
            node_actnum[rad_node] = min(k, actnum)           
#             if node_actnum[node] != 1:
#                 radiated_nodes.append(nodes)                      
        
        anchor_num  = len(anchors)
        # marginal score update
        for j in range(anchor_num, len(node_mscore)):
            nodej_mscore = 0                                           
            for rad_node in node_local_cluster[node_mscore[j][0]]:     
                actnum = node_actnum.get(rad_node, 0)                   
                nodej_mscore += (k - actnum)                            
            node_mscore[j] = (node_mscore[j][0], nodej_mscore)          
            
        node_mscore.sort(key = lambda x: x[1], reverse = True)
        anchors.append(node_mscore[i][0])
        
    end_time = time.time()                 
     
    execution_time = end_time - start_time
    print("Execuation time：", execution_time, "seconds")
    
#     print('anchor set:\n',anchors, len(anchors))
    return execution_time


def anchor_nodes_optmz_v1(graph, topk, k) -> []:
   
    nodes = list(graph.nodes())
    epsilon = 10 ** -2.5
    node_local_cluster = defaultdict(list)
    node_mscore:[(int, int)] =  []               # [(node, mscore)]

    for node in nodes:
        vec = forward_search(graph, node, epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0]
        node_local_cluster[node] = non_zero_indices            # indices of radiated neighbors
        node_mscore.append((node, k*len(non_zero_indices)))    # initial marginal score for each node
    
    start_time = time.time()                      
    node_actnum = {}
    anchors = []
    node_mscore.sort(key = lambda x: x[1], reverse = True)
    anchors.append(node_mscore[0][0])
    
    for i in range(1, topk):
         
        for rad_node in node_local_cluster[anchors[-1]]:               
            actnum = node_actnum.get(rad_node, 0)+1                    
            node_actnum[rad_node] = min(k, actnum)                     
             
        ## Step 1 : 找到第m个小于当前第二个的
          # 先更新marginal score对第i个点在node_mscore中
        nodei_mscore = 0       # 对其marginal score进行重新计算
        for rad_node in node_local_cluster[node_mscore[i][0]]:      
            actnum = node_actnum.get(rad_node, 0)
            nodei_mscore += (k - actnum)                            
        node_mscore[i] = (node_mscore[i][0], nodei_mscore)          
           
        m = i+1
        for j in range(i+1, len(node_mscore)):
            if node_mscore[j][1] < nodei_mscore:
                m = j
                break
        
        ## Step 2 : 重新更新排序
        for j in range(i+1, m+1):
            nodej_mscore = 0        
            for rad_node in node_local_cluster[node_mscore[j][0]]:          
                actnum = node_actnum.get(rad_node, 0)
                nodej_mscore += (k - actnum)                                
            node_mscore[j] = (node_mscore[j][0], nodej_mscore)              

        node_mscore[i:m] = sorted(node_mscore[i:m], key = lambda x: x[1], reverse = True)
        anchors.append(node_mscore[i][0])  
        
    
    end_time = time.time()
   
    execution_time = end_time - start_time
    print("Execuation time：", execution_time, "secondes")
    
#     print('Anchor Set:\n',anchors, len(anchors))
#     print([node[0] for node in node_mscore[0:len(anchors)]])
#     print([node[1] for node in node_mscore[0:len(anchors)]])
   
    return execution_time


def anchor_nodes(graph, topk, k) -> []:
    
    nodes = list(graph.nodes())
    epsilon = 10 ** -2
    node_local_cluster = defaultdict(list)
    for node in nodes:
        vec = forward_search(graph, node, epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0]
        node_local_cluster[node] = non_zero_indices            # indices of radiated neighbors
    
    start_time = time.time()                      
    node_actnum = {}
    node_mscore:[(int, int)] =  []                
    for node in nodes: 
        node_actnum[node] = 0
        node_mscore.append((node, k*len(node_local_cluster[node]))) # initial marginal score for each node
    
    anchors = []
    node_mscore.sort(key = lambda x: x[1], reverse = True)
    anchors.append(node_mscore[0][0])

    radiated_nodes = []    
    
    for i in range(1, topk):
         
        for node in node_local_cluster[anchors[-1]]:               
            node_actnum[node] = min(k, node_actnum[node]+1)       
            if node_actnum[node] != 1:
                radiated_nodes.append(nodes)                       
        
        anchor_num  = len(anchors)
         
        for j in range(anchor_num, len(node_mscore)):
            nodej_mscore = 0                                             
            for node in node_local_cluster[node_mscore[j][0]]:          
                nodej_mscore += (k - node_actnum[node])                 
            node_mscore[j] = (node_mscore[j][0], nodej_mscore)         
            
        node_mscore.sort(key = lambda x: x[1], reverse = True)
        anchors.append(node_mscore[i][0])
        
    end_time = time.time()                  
    
    execution_time = end_time - start_time
    print("Executation time：", execution_time, "secondes")
    
#     print('anchor set: ',anchors, len(anchors))
    return anchors


def anchor_nodes_optmz(graph, topk, k) -> []:
   
    nodes = list(graph.nodes())
    epsilon = 10 ** -2
    node_local_cluster = defaultdict(list)
    for node in nodes:
        vec = forward_search(graph, node, epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0]
        node_local_cluster[node] = non_zero_indices            # indices of radiated neighbors
    
    node_actnum = {}
    node_mscore:[(int, int)] =  []            # [(node, mscore)]
    for node in nodes: 
        node_actnum[node] = 0
        node_mscore.append((node, k*len(node_local_cluster[node]))) # initial marginal score for each node
    
    
    start_time = time.time()                  
    anchors = []
    node_mscore.sort(key = lambda x: x[1], reverse = True)
    anchors.append(node_mscore[0][0])

    radiated_nodes = []    
    
    for i in range(1, topk):
         
        for node in node_local_cluster[anchors[-1]]:               
            node_actnum[node] = min(k, node_actnum[node]+1)        
            if node_actnum[node] != 1:
                radiated_nodes.append(nodes)                       
        
        ## Step 1 :
         
        nodei_mscore = 0       #  marginal score recompute
        for node in node_local_cluster[node_mscore[i][0]]:          
            nodei_mscore += (k - node_actnum[node])                 
        node_mscore[i] = (node_mscore[i][0], nodei_mscore)          
           
        m = i+1
        for j in range(i+1, len(node_mscore)):
            if node_mscore[j][1] < nodei_mscore:
                m = j
                break
        
        ## Step 2 :
        if m == i+1:                                                
            anchors.append(node_mscore[i][0])
        else:        
            for j in range(i+1, m+1):
                nodej_mscore = 0        
                for node in node_local_cluster[node_mscore[j][0]]:          
                    nodej_mscore += (k - node_actnum[node])                 
                node_mscore[j] = (node_mscore[j][0], nodej_mscore)          
            
            node_mscore[i:m] = sorted(node_mscore[i:m], key = lambda x: x[1], reverse = True)
            anchors.append(node_mscore[i][0])
    
    end_time = time.time()
     
    execution_time = end_time - start_time
    print("Execuation time：", execution_time, "secondes")
    
#     print('anchor set:\n',anchors, len(anchors))
    return execution_time

  
def anchor_select_test()->None: 
    graph, labels = graph_read('citeseer')
    nodes = list(graph.nodes())
    print(type(nodes[0]))
    
    for k in [1,2,3,4,5]:
        print('K is:',k)
        for topk in [50, 100, 150, 200, 250]:
            print('Anchor scale is:', topk)
            runtime = anchor_nodes(graph, topk, k)
            opt_runtime = anchor_nodes_optmz(graph, topk, k)
            print('Optimize：', round(runtime/opt_runtime,2),'times')
            print()
#             runtime = anchor_nodes_v1(graph, topk, k)
#             opt_runtime = anchor_nodes_optmz_v1(graph, topk, k)
#             print('Optimize：', round(runtime/opt_runtime,2),'times')
    return


# anchor_select_test()

3: [1, 2]


# PPR

## Forward & Backward Search

In [45]:
def forward_search(graph, source_node, epsilon):
    alpha = 0.15
    nodes = list(graph.nodes)
    
    rf = {node: 0.0 for node in nodes}
    rf[source_node] = 1.0

    ppr_vector = {node: 0.0 for node in nodes}
    
    if graph.degree(source_node) == 0:
        ppr_vector[source_node] = 1.0
        return list(ppr_vector.values())
    
    queue = [source_node]
    
    iterate_num = 0
    while queue:
        iterate_num += 1
        for u in queue:
            if rf[u]/graph.degree(u) >= epsilon:
                for v in list(graph.neighbors(u)):
                    rf[v] += (1-alpha) * (rf[u]/graph.degree(u))
                    if rf[v]/graph.degree(v) >= epsilon and v not in queue:
                        queue.append(v)
                ppr_vector[u] += alpha * rf[u]
                rf[u] = 0
            else: 
                queue.remove(u)
    return list(ppr_vector.values())


def backward_search(graph, target_node, epsilon):
    alpha = 0.15
    nodes = list(graph.nodes)    

    rb = {node:0.0 for node in nodes}
    rb[target_node] = 1
    
    t_ppr_vec = {node :0.0 for node in nodes}
    
    if graph.degree(target_node) == 0:
        t_ppr_vec[target_node] = 1.0
        return list(t_ppr_vec.values())
    
    queue = [target_node]
    
    while queue:
        for v in queue:
            if rb[v] > epsilon: 
                for u in list(graph.neighbors(v)):
                    rb[u] += (1-alpha) * (rb[v]/graph.degree(u))
                    if rb[u] > epsilon and u not in queue:
                        queue.append(u)
                t_ppr_vec[v] += alpha * rb[v]
                rb[v] = 0
            else:
                queue.remove(v)
    
    return list(t_ppr_vec.values())



## PPR Vector Support 

In [46]:
def ppr_support(graph_name, epsilon = 1e-3):
    
    graph, _ = graph_read(graph_name)
    nodes = list(graph.nodes)
    n = len(nodes)
    epsilon = 1/n
    
    
     
    isolated_nodes = list(nx.isolates(graph))
    print('NO. of nodes is {}, NO. of isolated nodes is : {}'.format(len(nodes), len(isolated_nodes)))
    
#     random.shuffle(nodes)
    node_supports = []
    for node in nodes:
        vec = forward_search(graph, node, epsilon)
#         vec = backward_search(graph, node, epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0.0]
        if len(non_zero_indices) > 10:
            print(sum(vec))
#         print(len(non_zero_indices))
        node_supports.append(len(non_zero_indices))
    
     
    support_ave = sum(node_supports) / len(node_supports)
     
    median = statistics.median(node_supports)
     
    max_supprot, min_support = max(node_supports), min(node_supports)

    print('Support average:{}, median:{}, max:{}, min:{}'.format(support_ave,median,max_supprot,min_support))

    
    count = 0
    for num in node_supports:
        if num > 100:
            count += 1

    print(count)
    
    
    return


# ppr_support('wiki', epsilon = 1e-5)
# ppr_support('cora', epsilon = 1e-3)
# ppr_support('amac', epsilon = 1e-3)

# Node Embedding 

## RNEET

In [52]:
def rneet(graph, emb_dim = 50, K = 1, Theta = 2, Epsilon = 1e-2):
    
    n = graph.number_of_nodes()
    
    anchor_scale = 500
    repnodes, anchor_mscore = anchor_nodes_marginal(graph, topk = anchor_scale, k = K, epsilon = Epsilon)
    mscore_ave = [sum(anchor_mscore[i*20:i*20+20])/20 for i in range(int(anchor_scale/20))]
    node_num = first_less_theta_index(mscore_ave, Theta*K)*20+20     # 锚节点规模
    print('\tMarginal Score Average: {}'.format(mscore_ave))
    print('\tAnchor node scale: {}({:.2%}) with damping factor K: {}'.format(node_num, node_num/n, K))
    
    similarity_matrix = np.zeros((n, emb_dim+20))
    
    pagerank = np.array(list(nx.pagerank(graph).values()))
    for node in repnodes[0:emb_dim+20]:
        ##  Negative sampling(PR-based): x = log(x) / PR(x)
        pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
        pr_vec = pr_vec / pagerank                              
        pr_vec = np.nan_to_num(pr_vec, nan=0.0, posinf=0.0, neginf=0.0)
        vec_nce = np.where(pr_vec < 1, 0,                        # 如果 x 小于1，将log(x)设置为0
                      np.log(pr_vec ))                           # 否则设为x = log(x)
        similarity_matrix[:,repnodes.index(node)] = vec_nce
    
    iterations = int((node_num-(emb_dim+20))/20)
    print('Number of iterations:{}'.format(iterations))
    
    
    for i in range(0, iterations):
        print('Iteration {}'.format(i))
               
        mat_u, vec_sigma, mat_v = ln.svd(similarity_matrix, full_matrices=False)
        
        squared_sv_center = vec_sigma[math.floor(emb_dim)] ** 2
        # update sigma to shrink the row norms
        sigma_tilda = [(0.0 if d < 0.0 else math.sqrt(d)) for d in (vec_sigma ** 2 - squared_sv_center)]
#         print(sigma_tilda)
        # update matrix B where at 20 rows are all zero
        similarity_matrix = np.dot(mat_u, np.diagflat(sigma_tilda))
        
        for j in range(20):
#             print('Node index : {}'.format(i*emb_dim+j))
            node = repnodes[emb_dim + 20 + 20*i+j]
            ppr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
            pr_vec = pr_vec / pagerank                                
            pr_vec = np.nan_to_num(pr_vec, nan=0.0, posinf=0.0, neginf=0.0)
            vec_nce = np.where(pr_vec < 1, 0,                         
                          np.log(pr_vec ))                           
            similarity_matrix[:,emb_dim+j] = vec_nce
    
    mat_u, vec_sigma, mat_v = ln.svd(similarity_matrix, full_matrices=False)
    diag = np.power(vec_sigma[0 : emb_dim], 0.5)
    diag = diags(diag, 0)
    emb_mat = mat_u[:, 0 : emb_dim] @ diag
    
    return emb_mat


def rneet_test(graph_name = 'cora'):
    
    graph,_ = graph_read(graph_name)
    
    embeddings = rneet(graph, emb_dim = 50, K = 1, Theta = 6, Epsilon = 1e-2)
    print('Dimension is : {}'.format(embeddings.shape[1]))
#     print(embeddings[0])
    
    
    return 


# rneet_test(graph_name = 'cora')

# Evaluations

## Classification V.S. Negative Distribution

In [41]:
def cls_nd(graph_name: str, neg_dis = 'uniform', node_num = 50):

     
    graph, labels = graph_read(graph_name)
    nodes = list(graph.nodes())
    cls_num = len(np.unique(labels))
    labels_torch = torch.tensor(labels).long()
    n = graph.number_of_nodes()
    
     
    train_percent = 0.1
    test_percent = 1 - train_percent * 2
    train_num = int(n * train_percent)
    test_num  = int(n * test_percent )
        
    repnodes = nodes[:node_num]
    
    n, m = graph.number_of_nodes(), graph.number_of_edges()
    
     
    similarity_matrix = np.zeros((n, len(repnodes)))
    
    if neg_dis == 'no':
        print('Without Negative Sampling. ')
        for node in repnodes:
            pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
            similarity_matrix[:,repnodes.index(node)] = pr_vec
    
    elif neg_dis == 'uniform':
        print('Negative Samples Are Uniformly Distrubuted. ')
        for node in repnodes:
            pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
            ##  Negative sampling: x = log(x) + log(n) when x*n > 0
            vec_nce = np.where(pr_vec * n < 1, 0,          
                          np.log(pr_vec * n + 1e-20))      
            similarity_matrix[:,repnodes.index(node)] = vec_nce
    else:
        print('Negative Samples Are Biased Distrubuted. ')
         
        pagerank = np.array(list(nx.pagerank(graph).values()))
        for node in repnodes:
            ##  Negative sampling(PR-based): x = log(x) / PR(x)
            pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
            pr_vec = pr_vec / pagerank                    
            vec_nce = np.where(pr_vec < 1, 0,             
                          np.log(pr_vec ))                
            similarity_matrix[:,repnodes.index(node)] = vec_nce
        
    # SVD decomposition
    mat_u, vec_sigma, mat_v = ln.svd(similarity_matrix, full_matrices=False)
    d = 50
    diag = np.power(vec_sigma[0 : d], 0.5)
    diag = diags(diag, 0)
    embeddings = mat_u[:, 0 : d] @ diag
    dim = max(embeddings.shape[1], 50)
    node_embeddings = torch.tensor(embeddings).to(torch.float32)

    print(similarity_matrix.shape)
    
    similarity_matrix = torch.tensor(similarity_matrix).float()   
    
    labels_torch = torch.tensor(labels).long()
    cls_num = len(np.unique(labels))
    
    run_num = 10
    acc = 0
    for _ in tqdm(range(run_num), desc='Training Iterations',ncols=50):
        # Samples selection
        node_random_sort = [i for i in range(n)]   
        random.shuffle(node_random_sort)                                        
        ## train and test samples
        train_node = node_random_sort[0 : train_num]                            
        test_node = node_random_sort[train_num :train_num + test_num]            

        model = MultiLayerNet(input_size = dim, num_classes = cls_num, dropout_rate = 0.3)
        model.train()
        model_train(model, node_embeddings[train_node], labels_torch[train_node])
        model.eval()
        acc += accuracy(model, node_embeddings[test_node], labels_torch[test_node])
        
    print('Node classification accuracy: {:.2f}% with {} selected nodes'.
          format((acc/run_num)*100, len(repnodes)))

    return 

for num in [500, 1000, 1500, 2000, 2500, 7000]:
# for num in [2900]:
    print("*** "+"Node number is: "+num+" ***")
    graph_name = 'amap'
    cls_nd(graph_name, neg_dis = 'no', node_num = num)
    cls_nd(graph_name, neg_dis = 'uniform', node_num = num)
    cls_nd(graph_name, neg_dis = 'pr-based', node_num = num)
    print("*** "+ "Finished!"+ " ***")

## Visualization

In [54]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib.patches import Rectangle

def visualization(graph_name = 'cora', node_num = 10, neg_dis = 'no', rdm = 'True'):

   
    graph, labels = graph_read(graph_name)
    nodes = list(graph.nodes())
    cls_num = len(np.unique(labels))
    labels_torch = torch.tensor(labels).long()
    n = graph.number_of_nodes()
    
    if rdm == 'True':
#         repnodes = ordered_list_centrality_probability(graph, "pr")[-50:]
        repnodes = random.sample(nodes, 50)
        
    else:
        repnodes = anchor_nodes(graph, 50, 1)
#         repnodes = ordered_list_centrality_probability(graph, "pr")[0:50]
#         repnodes = ordered_list_centrality_probability(graph, "pr")[0:node_num]
    
    
    n, m = graph.number_of_nodes(), graph.number_of_edges()
    
    
    similarity_matrix = np.zeros((n, len(repnodes)))
    
    
    if neg_dis == 'no':
        print('Without Negative Sampling. ')
        for node in repnodes:
            pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
            similarity_matrix[:,repnodes.index(node)] = pr_vec
    
    elif neg_dis == 'uniform':
        print('Negative Samples Are Uniformly Distrubuted. ')
        for node in repnodes:
            pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
            ##  Negative sampling: x = log(x) + log(n) when x*n > 0
            vec_nce = np.where(pr_vec * n < 1, 0,         
                          np.log(pr_vec * n + 1e-20))     
            similarity_matrix[:,repnodes.index(node)] = vec_nce
    else:
        print('Negative Samples Are Biased Distrubuted. ')
         
        pagerank = np.array(list(nx.pagerank(graph).values()))
        for node in repnodes:
            ##  Negative sampling(PR-based): x = log(x) / PR(x)
            pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
            pr_vec = pr_vec / pagerank                   
            vec_nce = np.where(pr_vec < 1, 0,            
                          np.log(pr_vec ))               
            similarity_matrix[:,repnodes.index(node)] = vec_nce
        
    mat_u, vec_sigma, mat_v = ln.svd(similarity_matrix, full_matrices=False)
    
    dimension = 50
    diag = np.power(vec_sigma[0 : dimension], 0.5)
    diag = diags(diag, 0)
    embeddings_matrix = mat_u[:, 0 : dimension] @ diag
    tsne = TSNE(n_components=2, learning_rate='auto', random_state=42)
    X_2d = tsne.fit_transform(embeddings_matrix)
    
#     pca = PCA(n_components=2)
#     X_2d = pca.fit_transform(embeddings_matrix)
    fig = plt.figure(figsize=(3, 3))
#     plt.scatter(X_2d[:, 0], X_2d[:, 1], c=labels, cmap='jet', s=0.5)
    
   
    random_indices = random.sample(range(len(X_2d)), 1000)
    random_points = X_2d[random_indices]
    random_labels = labels[random_indices]
    plt.scatter(random_points[:, 0], random_points[:, 1], c=random_labels, cmap='jet', s=2)  # cmap为色图选择Jet

    
    # 隐藏横纵坐标
    plt.xticks([])
    plt.yticks([])

#     plt.colorbar()
    
    plt.tight_layout()

    plt.show()
    
    return



## Radiation marginal and performance marginal

In [26]:
def anchor_nodes_marginal(graph, topk, k, epsilon = 10 ** -2) -> []:
    
    nodes = list(graph.nodes())
#     epsilon = 10 ** -2
    node_local_cluster = defaultdict(list)
    for node in nodes:
        vec = forward_search(graph, node, epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0]
        node_local_cluster[node] = non_zero_indices            # indices of radiated neighbors
    
    start_time = time.time()                      
    node_actnum = {}
    node_mscore:[(int, int)] =  []               # [(node, mscore)]
    for node in nodes: 
        node_actnum[node] = 0
        node_mscore.append((node, k*len(node_local_cluster[node]))) # initial marginal score for each node
    
    anchors = []
    node_mscore.sort(key = lambda x: x[1], reverse = True)
    anchors.append(node_mscore[0][0])

    anchor_mscore = []
    anchor_mscore.append(node_mscore[0][1])
    
    radiated_nodes = []   
    
    for i in range(1, topk):
         
        for node in node_local_cluster[anchors[-1]]:              
            node_actnum[node] = min(k, node_actnum[node]+1)        
            if node_actnum[node] != 1:
                radiated_nodes.append(nodes)                       
        
        anchor_num  = len(anchors)
         
        for j in range(anchor_num, len(node_mscore)):
            nodej_mscore = 0                                             
            for node in node_local_cluster[node_mscore[j][0]]:          
                nodej_mscore += (k - node_actnum[node])                 
            node_mscore[j] = (node_mscore[j][0], nodej_mscore)          
            
        node_mscore.sort(key = lambda x: x[1], reverse = True)
        anchors.append(node_mscore[i][0])
        anchor_mscore.append(node_mscore[i][1])
        
    end_time = time.time()                  
     
    execution_time = end_time - start_time
    print("\tExecuation：", execution_time, "secondes")
    
#     print('anchor set: ',anchors, len(anchors))
    return anchors, anchor_mscore


def first_zero_index(lst):
    for i, item in enumerate(lst):
        if item == 0:
            return i
    return None  # 如果列表中没有非零项，则返回 None


def first_less_theta_index(lst, theta):
    for i, item in enumerate(lst):
        if item <= theta:
            return i
    return None   


def marginal_effect_performance(graph, labels, K = 1, Theta = 15, anchor_scale = 500, Epsilon = 10**-2):
     
    nodes = list(graph.nodes())
    cls_num = len(np.unique(labels))
    labels_torch = torch.tensor(labels).long()
    n = graph.number_of_nodes()
    
     
    train_percent = 0.1
    test_percent = 1 - train_percent * 2
    train_num = int(n * train_percent)
    test_num  = int(n * test_percent )
    
    repnodes, anchor_mscore = anchor_nodes_marginal(graph, topk = anchor_scale, k = K, epsilon = Epsilon)
    mscore_ave = [sum(anchor_mscore[i*20:i*20+20])/20 for i in range(int(anchor_scale/20))]
    node_num = first_less_theta_index(mscore_ave, Theta*K)*20+20      
    print('\tMarginal Score Average: {}'.format(mscore_ave))
    print('\tAnchor node scale: {}({:.2%}) with damping factor K: {}'.format(node_num, node_num/n, K))
    
    node_local_cluster = defaultdict(list)
    for node in nodes:
        vec = forward_search(graph, node, epsilon = Epsilon)
        non_zero_indices = [index for index, value in enumerate(vec) if value != 0]
        node_local_cluster[node] = non_zero_indices               # indices of radiated neighbors
    
    coverage = []
    cover_margin = []
    performance = [] 
    performance_margin = []
    for node in repnodes[0: node_num]:
        coverage = list(set(coverage) | set(node_local_cluster[node]))
    print("\tTotal {}({:.2%}) nodes are covered.".format(len(coverage), len(coverage)/n))
    
     
    similarity_matrix = np.zeros((n, node_num))

    
    pagerank = np.array(list(nx.pagerank(graph).values()))
    for node in repnodes[0:node_num]:
        ##  Negative sampling(PR-based): x = log(x) / PR(x)
        pr_vec = np.array(list(nx.pagerank(graph, personalization={node: 1}).values()))
        pr_vec = pr_vec / pagerank                                
        vec_nce = np.where(pr_vec < 1, 0,                         
                      np.log(pr_vec ))                            
        similarity_matrix[:,repnodes.index(node)] = vec_nce
        
    # SVD decomposition
    mat_u, vec_sigma, mat_v = ln.svd(similarity_matrix, full_matrices=False)
    d = 50 if node_num >= 50 else node_num 
    diag = np.power(vec_sigma[0 : d], 0.5)
    diag = diags(diag, 0)
    embeddings = mat_u[:, 0 : d] @ diag
    node_embeddings = torch.tensor(embeddings).to(torch.float32)
#     print(similarity_matrix.shape)
    
    similarity_matrix = torch.tensor(similarity_matrix).float()   
    labels_torch = torch.tensor(labels).long()
    cls_num = len(np.unique(labels))
    
    run_num = 20
    acc = 0
    for _ in tqdm(range(run_num), desc='\tTraining Iterations',ncols=50):
        # Samples selection
        node_random_sort = [i for i in range(n)]   
        random.shuffle(node_random_sort)                                         
        ## train and test samples
        train_node = node_random_sort[0 : train_num]                             
        test_node = node_random_sort[train_num :train_num + test_num]            

        model = MultiLayerNet(input_size = d, num_classes = cls_num, dropout_rate = 0.3)
        model.train()
        model_train(model, node_embeddings[train_node], labels_torch[train_node])
        model.eval()
        acc += accuracy(model, node_embeddings[test_node], labels_torch[test_node])
        
    print('\tNode classification accuracy: {:.2f}% with K = {}, threshold theta {}'.format((acc/run_num)*100, K, Theta))
    
    return round((acc/run_num)*100, 2)


def anchor_nodes_marginal_test(graph_name):
    
     
    graph, labels = graph_read(graph_name)
    
    anchors, anchor_mscore = anchor_nodes_marginal(graph, 500, 2)
#     print(anchor_mscore)
    
    mscore_ave = [sum(anchor_mscore[i*20:i*20+20])/20 for i in range(25)]
    print()
    print(mscore_ave)
    
    print(first_less_theta_index(mscore_ave, 5)*20)
    
    return
# anchor_nodes_marginal_test('cora')

def marginal_effect_performance_eval(graph_name):
    graph, lables = graph_read(graph_name)
    Ks = [1, 2, 3, 4, 5]
    Thetas = [16, 14, 12, 10, 8, 6, 4, 2]
#     Ks = [1, 2]
#     Thetas = [16, 14]
    
    result_mat = [[0] * len(Thetas) for _ in range(len(Ks))]
    
    for k in Ks:
        print('\n**** Damping factor K is {} **** '.format(k))
        i = Ks.index(k)
        for theta in Thetas:
            j = Thetas.index(theta)
            print('    ** Termination threshold theta is {} **'.format(theta))
            result_mat[i][j] = marginal_effect_performance(graph, lables, K = k, Theta = theta, anchor_scale = 2000, Epsilon = 10**-3)
        print() 
    
    print(result_mat)
    for i, row in enumerate(result_mat):
        print("K is: {}, accuracy is: {} .".format(i, row))

    custom_x_ticks = [16, 14, 12, 10, 8, 6, 4, 2]
#     custom_x_ticks = [16, 14]
    
    for i, row in enumerate(result_mat):
        plt.plot(row, label=f'K =  {i+1}')

#     plt.title('Five Lines Plot')
    plt.xlabel(graph_name.capitalize()+' Threshold '+r'$\theta$')
    plt.ylabel('Accuracy')

    plt.xticks(range(len(custom_x_ticks)), custom_x_ticks)
    
    plt.legend()
    plt.show()
    
    return


# marginal_effect_performance_eval('cora')
# marginal_effect_performance_eval('wiki')
# marginal_effect_performance_eval('acm')
# marginal_effect_performance_eval('citeseer')
# marginal_effect_performance_eval('amap')    # epsilon = 10 **-3

## Link Prediction

In [55]:
import itertools
from itertools import combinations

def link_prediction(graph_name):
    
    
    graph, _ = graph_read(graph_name)
    
    # edges selection for training and testing
    nodes = list(graph.nodes)
    edges = list(graph.edges) 
    num = int(len(edges)*0.2)
    cls_num = 2
    # node pairs selection
    random.shuffle(edges)
    pair_edges = edges[:num] # random selection
    pair_non_edges = random.sample([p for p in itertools.combinations(nodes,2) if p not in graph.edges],num) 
    print('Pair selection finished!')
    
    # Samples selection
    train_percent = 0.5
    test_percent = 1 - train_percent
    pair_num = num *2
    train_num = int(pair_num * train_percent)
    test_num  = int(pair_num * test_percent )
    
    pair_random_sort = [i for i in range(num*2)]   
    random.shuffle(pair_random_sort)                                           
    ## train sample
    train_pair = pair_random_sort[0 : train_num]                                
    test_pair = pair_random_sort[train_num : (train_num + test_num)]            
    
    
    embeddings_matrix = rneet(graph, emb_dim = 50, K = 1, Theta = 14, Epsilon = 1e-2)
    
    
    emb_dim = embeddings_matrix.shape[1]
    
    # Concatenate 
    pair_matrix = np.zeros((num*2, emb_dim*2))
    is_edge = []
    for i in range(num):
        u_idx, v_idx = nodes.index(pair_edges[i][0]), nodes.index(pair_edges[i][1])
        is_edge.append(1)
        pair_matrix[2*i][0:emb_dim] = embeddings_matrix[u_idx]
        pair_matrix[2*i][emb_dim:emb_dim*2] = embeddings_matrix[v_idx]

        u_idx, v_idx = nodes.index(pair_non_edges[i][0]), nodes.index(pair_non_edges[i][1])
        is_edge.append(0)
        pair_matrix[2*i+1][0:emb_dim] = embeddings_matrix[u_idx]
        pair_matrix[2*i+1][emb_dim:emb_dim*2] = embeddings_matrix[v_idx]
    
    pair_dim = pair_matrix.shape[1]
    print('Link dimension is : {}'.format(pair_dim))
    
    pair_embeddings = torch.tensor(pair_matrix).to(torch.float32)
    labels_torch = torch.tensor(is_edge).long()
    run_num, acc = 10, 0.0
    for _ in range(run_num):
        model = MultiLayerNet(input_size = pair_dim, num_classes = cls_num, dropout_rate=0.3)
        model.train()
        model_train(model,pair_embeddings[train_pair], labels_torch[train_pair])
        model.eval()
        acc += accuracy(model, pair_embeddings[test_pair], labels_torch[test_pair])
    
    appro_acc = round(acc/run_num, 4)
    print('Link prediction accuracy :{} with Concatenate'.format(appro_acc))
            
    # Hadmard Product                                              
    pair_matrix = np.zeros((num*2, emb_dim))
    is_edge = []
    for i in range(num):
        u_idx, v_idx = nodes.index(pair_edges[i][0]), nodes.index(pair_edges[i][1])
        is_edge.append(1)
        pair_matrix[2*i] = np.multiply(embeddings_matrix[u_idx], embeddings_matrix[v_idx])

        u_idx, v_idx = nodes.index(pair_non_edges[i][0]), nodes.index(pair_non_edges[i][1])
        is_edge.append(0)
        pair_matrix[2*i+1] = np.multiply(embeddings_matrix[u_idx], embeddings_matrix[v_idx])
            
    pair_dim = pair_matrix.shape[1]
    print('Link dimension is : {}'.format(pair_dim))
    
    pair_embeddings = torch.tensor(pair_matrix).to(torch.float32)
    labels_torch = torch.tensor(is_edge).long()
    run_num, acc = 10, 0.0
    for _ in range(run_num):
        model = MultiLayerNet(input_size = pair_dim, num_classes = cls_num, dropout_rate=0.3)
        model.train()
        model_train(model,pair_embeddings[train_pair], labels_torch[train_pair])
        model.eval()
        acc += accuracy(model, pair_embeddings[test_pair], labels_torch[test_pair])
    
    appro_acc = round(acc/run_num, 4)
    print('Link prediction accuracy :{} with Hardamard Product'.format(appro_acc))
    
    
    return


def link_prediction_test():
    
    link_prediction('cora')
    
    
    return
